

### Course Number: INFO 7374
### Assignment: 5


---


### *Student Name: Krishnakanth Naik Jarapala*
### *NU-ID: 002724795*



### 1. Import Required Libraries

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv() 

import re
import time
from langchain_core import tools, prompts
from langchain.chains import LLMChain, SequentialChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI, ChatAnthropic
from langchain.tools import BaseTool, StructuredTool, Tool
from langchain.memory import SimpleMemory
from langchain_community.llms import Ollama
from langchain_community.tools import BraveSearch
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\jkkn7\PycharmProjects\Research-Agent-langchain\raagentvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Intialize your LLM

In [2]:
## Function to LLM Model
def load_llm_model(model_name):

    if model_name == 'gemini':
        return ChatGoogleGenerativeAI(model="gemini-pro",
                             verbrose= True,
                             temperature= 0.8,
                             google_api_key= os.getenv('GEMINI_API_KEY'))
    if model_name == 'local':
        return Ollama(model="llama3")


## Load Required Model
llm = load_llm_model('local')

In [3]:
## Lets Check Model response
print(llm.invoke("Write 2 sentence about Machine Learning?"))

Here are two sentences about machine learning:

Machine learning is a subfield of artificial intelligence that involves training computers to make predictions or decisions based on patterns and relationships within large datasets. By analyzing data, identifying trends, and adapting to new information, machine learning models can improve their performance over time, enabling applications such as image recognition, natural language processing, and predictive analytics.


## Tools

#### 1. Topic Breakdown Tool
* uses Gemini LLM or Local Model to break the main topic into smaller, more focused subtopics.

In [4]:
# Topic Breakdown Tool
topic_breakdown_prompt = ChatPromptTemplate.from_template(
    '''
    Break down the research topic: '{topic}' into 2-3 diverse subtopics, output should have only the list of subtopics
    '''
)
topic_breakdown_chain = LLMChain(
    llm=llm, 
    prompt=topic_breakdown_prompt, 
    output_key="subtopic"  # The output variable for this chain
)

c:\Users\jkkn7\PycharmProjects\Research-Agent-langchain\raagentvenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [5]:
# sample Topic
topic_name = "AI in Healthcare"

# Run the Topic Breakdown Chain
subtopics = topic_breakdown_chain.invoke(input = {'topic': topic_name})
print(subtopics['subtopic'])

Here are 3 diverse subtopics related to "AI in Healthcare":

• **Predictive Modeling and Disease Diagnosis**
• **Personalized Medicine and Treatment Optimization**
• **Clinical Decision Support Systems and Patient Engagement**


### 2. Query Expansion Tool
* Generates related keywords, synonyms, and phrases on expanded the sub-topic generated by the Topic Breakdown Tool to enhance the search results.

In [6]:
# Query Expansion Tool
query_expansion_prompt = ChatPromptTemplate.from_template(
    "Generate only 3 related phrases for the subtopic '{subtopic}', output should have only the list of phrases"
)

query_expansion_chain = LLMChain(
    llm=llm, 
    prompt=query_expansion_prompt, 
    output_key="expanded_queries"  
)

In [7]:
# Run the Query Expansion Chain
expanded_topics = query_expansion_chain.invoke(input = {'subtopic': subtopics['subtopic']})
print(expanded_topics['expanded_queries'])


Here are three related phrases for the subtopic 'Here are 3 diverse subtopics related to "AI in Healthcare"':

• AI-Driven Diagnostic Accuracies
• Tailored Therapies through Data Analytics
• Intelligent Guidance for Healthier Choices


### 3. Search Tool - Google Serper API

In [8]:
from langchain.utilities import GoogleSerperAPIWrapper

os.environ["SERPER_API_KEY"] = os.getenv('SERPER_API_KEY')

search = GoogleSerperAPIWrapper( )

tool = Tool(
    name="Scrape google searches",
    func=search.run,
    description="useful for when you need to ask the agent to search the internet",
)

In [9]:
def run_search(topic, expanded_queries):
     # Conduct the initial search on the main topic
    main_search_results = tool.run(topic)
    time.sleep(2)  # Pause to prevent rate limiting or overload
    
    additional_search_results = ''
    
    # Extract individual queries using regular expressions
    queries = re.findall(r'\+\s*"([^"]+)"', expanded_queries)
    
    # Perform searches for each extracted query
    for query in queries:
        time.sleep(2)  # Pause between searches
        result = tool.run(query)  # Fetch search results for the current query
        additional_search_results += result  # Append the result to the cumulative string
    
    # Combine the results from the main topic search and the expanded queries
    combined_results = main_search_results + additional_search_results
    
    return combined_results

In [10]:
search_results = run_search(topic_name, expanded_topics['expanded_queries'])
print(search_results)

Given the rapid advances in AI for imaging analysis, it seems likely that most radiology and pathology images will be examined at some point by a machine. Healthcare AI systems can analyze patterns in a patient's medical history and current health data to predict potential health risks. This predictive capability ... AI can be used to support digital communications, offering schedule reminders, tailored health tips and suggested next steps to patients. The ability of AI to ... When it comes to supporting the overall health of a population, AI can help people manage chronic illnesses themselves — think asthma, diabetes ... What you'll learn · Identify problems healthcare providers face that machine learning can solve · Analyze how AI affects patient care safety, quality, and ... Additionally, AI increases the ability for healthcare professionals to better understand the day-to-day patterns and needs of the people they care for, and with ... AI is helping doctors analyze images more quic

### 4. Summarize Search Results Tool
* Summarize the provided search results on the topic

In [11]:
# Search Summary Tool
summary_prompt = ChatPromptTemplate.from_template(
    '''
    Summarize the provided search results on the topic: {topic}. 
    Make sure to include all the important points, themes, and details, presenting the information in a well-organized and easy-to-understand way. 
    The summary should offer a clear, high-level overview that encapsulates the core insights from the search results. 
    Please ensure that no factual information is modified.
    Search Results are: {search_results} 
    '''
)

summary_search_results_chain = LLMChain(
    llm=llm, 
    prompt=summary_prompt, 
    output_key="summary"  
)

In [12]:
summary = summary_search_results_chain.run(topic=topic_name,search_results=search_results)
print(summary)

c:\Users\jkkn7\PycharmProjects\Research-Agent-langchain\raagentvenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


**AI in Healthcare: A Comprehensive Summary**

The search results highlight the significant impact of Artificial Intelligence (AI) on the healthcare industry, with a focus on imaging analysis, predictive capabilities, patient communication, chronic illness management, and overall population health.

**Imaging Analysis and Pattern Detection**

AI can analyze radiology and pathology images to identify patterns and detect potential health risks. This technology is likely to become increasingly prevalent in radiology and pathology departments.

**Predictive Capabilities and Patient Safety**

Healthcare AI systems can analyze a patient's medical history and current data to predict potential health risks, enabling early intervention and improved patient outcomes. Additionally, AI increases the ability for healthcare professionals to better understand patients' day-to-day patterns and needs.

**Patient Communication and Support**

AI-powered digital communications can provide schedule reminde

### 5. Critique Tool
* The tool critiques the summary, and offers suggestions of how to improve and potentially other relevant topics to search for. 

In [13]:
# 4. Critique Tool
critique_prompt = ChatPromptTemplate.from_template(
    '''
    Here is a summary of search results: {summary}, covering different aspects and subtopics of {topic}.
    Review this summary by critiques and pointing out any omissions, errors, or sections that could benefit from further refinement. 
    Ensure that the facts remain unchanged.
    Following your critique, produce a revised summary that resolves the issues identified, improving the clarity, thoroughness, and overall quality of the content.
    Output should have only the revised summary.
    '''
)
critique_chain = LLMChain(
    llm=llm, 
    prompt=critique_prompt, 
    output_key="critique"  
)

In [14]:
critique = critique_chain.run(summary=summary, topic=topic_name)
print(critique)

**Revised Summary: AI in Healthcare - A Comprehensive Review**

The search results highlight the transformative impact of Artificial Intelligence (AI) on the healthcare industry, covering imaging analysis, predictive capabilities, patient communication, chronic illness management, population health, automation, and generative AI.

**Imaging Analysis and Pattern Detection**

AI-powered image analysis enhances radiology and pathology departments by detecting patterns and identifying potential health risks. This technology will become increasingly prevalent in medical imaging.

**Predictive Capabilities and Patient Safety**

Healthcare AI systems analyze patient data to predict potential health risks, enabling early intervention and improved outcomes. AI also enables healthcare professionals to understand patients' day-to-day needs and patterns.

**Patient Communication and Support**

AI-powered digital communications provide personalized reminders, health tips, and next steps, enhancing 

### Complete Workflow - `Research Assistant as a Agent`

In [15]:
# Function to run the agent
def run_research_agent(topic: str):

    # Agent - Topic Break Down + Query Expansion
    query_expansion_agent = SequentialChain(
        chains=[
            topic_breakdown_chain,
            query_expansion_chain,
            ],
        input_variables=["topic"],  # Initial input to the chain
        output_variables=["expanded_queries"],  # Final output of the chain
        verbose=True
    )
    
    # Topic Break Down + Query Expansion
    expanded_queries = query_expansion_agent.run({"topic": topic})
    
    # Search Tool - on Expanded Queries
    search_results = run_search(topic, expanded_queries) 

    # Search Summary Tool
    summary = summary_search_results_chain.run(topic=topic, search_results=search_results)    

    # Critique and Improve Results
    critique = critique_chain.run(summary=summary, topic=topic)
    
    return critique

### Research Assistant Agent - Improved Summary Results

In [18]:
### Topic to do reasearch!
topic = "AI in Agriculture"
improved_agent_summary = run_research_agent(topic)
print(improved_agent_summary)



> Entering new SequentialChain chain...

> Finished chain.
**Revised Summary: AI in Agriculture**

The application of Artificial Intelligence (AI) in agriculture is revolutionizing the industry by providing farmers with real-time insights, reducing labor and input costs, and improving yields. The technology offers a range of benefits, including:

* Real-time crop monitoring: AI provides farmers with valuable insights on crop health, growth, and development, enabling them to identify areas that require irrigation, fertilization, or pest control.
* Labor and input cost reduction: By automating tasks such as crop monitoring, AI can reduce labor costs and minimize the use of inputs like water, fertilizers, and pesticides.
* Improved yields: AI-powered precision agriculture enables farmers to optimize growing conditions, leading to higher yields and better quality crops.

The three key challenges facing agriculture today – increasing global demand for food, climate change, and environment

### Simple LLM Response on Same Topic

In [19]:
## LLM Model Response on Same Topic
simple_llm_response = llm.invoke(topic)
print(simple_llm_response)

Artificial intelligence (AI) is transforming the agriculture industry in various ways, from improving crop yields to enhancing decision-making processes. Here are some examples of AI in agriculture:

1. **Precision Farming**: AI-powered precision farming uses sensors, drones, and satellites to monitor soil health, weather conditions, and crop growth. This data is used to optimize planting, irrigation, and harvesting decisions.
2. **Crop Prediction**: AI algorithms analyze satellite and sensor data to predict crop yields, detect early signs of pests and diseases, and provide insights on optimal harvesting times.
3. **Autonomous Farming**: Self-driving tractors and autonomous farming systems use AI to navigate fields, plant seeds, and apply fertilizers and pesticides with precision.
4. **Livestock Monitoring**: AI-powered sensors and drones monitor livestock health, detecting early signs of illness or stress. This enables farmers to take proactive measures to improve animal welfare and p

# Conclusion

* It is evident that the output from the Simple LLM model is vague, generic, and contains factual inconsistencies. In contrast, our Research Agent effectively addresses these issues, delivering more precise and reliable results.